<a href="https://colab.research.google.com/github/jcklpe/neural-art-scripts/blob/master/neural_style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INTRO**

Hi, this is a script for installing and running a style transfer neural network in Google's cloud-based collaboratory notebooks. 

You can also copy and paste these commands into the terminal of your local Linux machine to get the same basic set up. You'll need to install CUDA and CUDNN to your local network first though. 

You can run each of the commands by selecting the code cell and clicking the play button. 

### **SETUP**

#### **Verify NVIDIA card is hooked up to instance.**

In [ ]:
!nvidia-smi

If it's not you need to edit the runtime up in the top menu. 

#### **Import OS library package**
This will make it easier to do file system manipulations in python script. We could (and will for some things)run bash subprocesses using the `!` but hey, I thought this would be a fun project to teach myself more python ay!

In [ ]:
import os

#### **Define project files root**
We've written things this way so that it's more portable. 

In [ ]:
projectFiles = "/project-files"
os.makedirs(projectFiles, exist_ok=True)


#### **Mount your google drive to handle your project files**
When you run this it will bring up a link and a form input. Click the link to open an authorization flow to your Google Drive. Then copy the authorization code and paste it into the form. 

You can see the files in the collaboratory instance file system by clicking the folder icon to the left.

In [ ]:
from google.colab import drive
drive.mount(f"{projectFiles}/gdrive")

#### **Set up symlink to Google Drive**
The mounted Google Drive contains the folder "My Drive" which causes problems for running the scripts due to the space, so we're going to get around that by symlinking to skip that nasty path.

In [ ]:
!ln -sfn '/project-files/gdrive/My Drive/neural-network' {projectFiles}/symdrive

In [ ]:
symDrive = f'{projectFiles}/symdrive'

### **Create project variables for project file structure**
Creating these variables abstracts the file structure a little bit. It will make more sense in just a sec. 

In [ ]:
os.makedirs(f"{projectFiles}/gdrive/My Drive/neural-network", exist_ok=True)
imageLibrary = f"{symDrive}/image-library"
contentDir = f"{symDrive}/content"
styleDir = f"{symDrive}/style"
outputDir = f"{symDrive}/output"
neuralScript = f"{projectFiles}/neural-style-pt/neural_style.py"

#### **Create project folders**
We're going to create a neural-network project folder with 4 sub folders for your project. 

1. The image-library directory holds all the images you plan on using, both content and style images. Lots of images can be used as both so we're going to keep our reference image library in a flat structure. We'll also create a variable for the directories we create for future use.

In [ ]:
os.makedirs(imageLibrary, exist_ok=True)

2. This script runs as a batch process so you can process a lot of images and styles automatically. Copy/paste the images you want to run into the content and style folders. These will be cross referenced against each other and exported to the output folder. 

In [ ]:
os.makedirs(contentDir, exist_ok=True)
os.makedirs(styleDir, exist_ok=True)

3. And the aformentioned output folder.

In [ ]:
os.makedirs(outputDir, exist_ok=True)


#### **Git clone the neural style project, and install the python dependencies**

In [ ]:
os.chdir(projectFiles)
!git clone https://github.com/ProGamerGov/neural-style-pt.git 
!pip install neural-style
!pip install torch torchvision


#### **Git clone pytorch and then run it's install script**

In [ ]:
!git clone --recursive https://github.com/pytorch/pytorch
os.chdir(f"{projectFiles}/pytorch")
!python setup.py install

#### **Using the neural-style python project installed via pip, install the models into the neural-style-pt/models**

In [ ]:
!neural-style -download_models /project-files/neural-style-pt/models/

#### **Define batch process function**

In [ ]:
def batchNeural(contentImages, styleImages):
  # move to neural-styles-pt folder context to properly run the command
  os.chdir(f"{projectFiles}/neural-style-pt")

  # iterate through all files in the content folder
  for contentEntry in os.scandir(contentImages):

    # isolate the name of the content image without the extension or path and set to variable.  
    contentFileName = os.path.splitext(contentEntry.name)[0] 

    # make a folder to contain all output of a content kind 
    os.makedirs(f"{outputDir}/{contentFileName}", exist_ok=True) 

    # iterate through all files in the style folder   
    for styleEntry in os.scandir(styleImages):

      # isolate the name of the style image without the extension or path and set to variable.
      styleFileName = os.path.splitext(styleEntry.name)[0]
      !python {neuralScript} -style_image {styleDir}/{styleEntry.name} -content_image {contentDir}/{contentEntry.name} -save_iter 400 -image_size 600 -num_iterations 1200 -backend cudnn -output_image {outputDir}/{contentFileName}/{styleFileName}-{contentFileName}.png 
      print("===================")
      print(f"The '{styleFileName}' style was applied to '{contentFileName}' content image")
      print("======================================")


### **RUN THE SCRIPT!**
Make sure your files are set up in the right folders and invoke the function

In [ ]:
batchNeural(contentDir, styleDir)